In [ ]:
# Uses the growth-curve.xlxs data 
!pip install pandas matplotlib
# intall openpyxl to open excel 
!pip install openpyxl 
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import from excel using pandas 
gc_data = pd.read_excel('growth-curve.xlsx', header=0)
print(gc_data)

print(gc_data['Timepoint'][0:3]) # check the header information is working ok...


In [ ]:
#Plot all the data

import matplotlib.pyplot as plt

plt.figure(figsize=(5, 3))                
plt.scatter(gc_data['Timepoint'], gc_data['OD_true'])            

# Titles and labels
plt.xlabel('Timepoint')
plt.ylabel('OD_true')
plt.title('Example growth curve')

# Show your figure
plt.show()

In [ ]:
#plot selection based on 'Treatment' = UNT

#use pandas .loc() to select only those data where Treatment = UNT

UNT_data = gc_data.loc[gc_data['Treatment'] == 'UNT']
print(UNT_data[0:3])

plt.figure(figsize=(5, 3))                
plt.scatter(UNT_data['Timepoint'], UNT_data['OD_true'])            

# Titles and labels
plt.xlabel('Timepoint')
plt.ylabel('OD_true')
plt.title('Growth curve for Treatment = UNT')

# Show your figure
plt.show()


In [ ]:
# Calculate doubling time based on:
# https://www.doubling-time.com/compute_more.php

import numpy as np
from scipy.optimize import curve_fit

# define a fitting fuction, which is a sine curve with a few extra fitting parameters
def fit(x, a, b):  
    return a * np.exp(b*x)

xfit = np.linspace(min(UNT_data['Timepoint']), 1.1*max(UNT_data['Timepoint']), num=101)  

guess = [1,1] # initial guesss for the fitting parameters

popt, pcov = curve_fit(fit, UNT_data['Timepoint'], UNT_data['OD_true'], guess)
errors = np.sqrt(np.diag(pcov))

plt.figure(figsize=(5, 3))  
plt.scatter(UNT_data['Timepoint'], UNT_data['OD_true'],c="red")
plt.plot(xfit, fit(xfit, *popt), 'g--', label='fit: a=%5.3f, b=%5.3ff' % tuple(popt))
plt.legend(loc='upper right')
plt.xlabel('Timepoint')
plt.ylabel('OD_true')
plt.title('Growth curve and exponential fit')
plt.xlim(min(UNT_data['Timepoint']), 1.1*max(UNT_data['Timepoint'])) # Set x-axis
plt.show()

print('fit: a=%5.4f, b=%5.4f' % tuple(popt))
print('errors: a=%5.4f, b=%5.4f' % tuple(errors))
print()

doubling_time = np.log(2)/(popt[1])

print(f'Growth Rate = {popt[1]: .4f}')
print(f'At t=0, calculated cell concentration = {popt[0]: .4f}')
print(f'Doubling time = {doubling_time: .4f}')

In [ ]:
# Laura suggested using the 'groupby' function to separate the data by Treatment 
# We then loops through each treatment, fitting and plotting each group of data

import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Import data from excel using pandas
gc_data = pd.read_excel('growth-curve.xlsx', header=0)

#group by treatment type
groups = gc_data.groupby('Treatment')

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#define exponential fitting function
def fit(x, a, b):
    return a * np.exp(b*x)

print("Doubling time for:")

#loop to fit and plot each treatment group separately
for name, group in groups:
    #fitting exponential curve
    xfit = np.linspace(min(group.Timepoint), 1.1*max(group.Timepoint), num=101)
    guess = [1,1] # initial guesss for the fitting parameters
    popt, pcov = curve_fit(fit, group.Timepoint, group.OD_true, guess)
    errors = np.sqrt(np.diag(pcov))

    #plot the points and capture the assigned marker colour
    line, = ax.plot(group.Timepoint, group.OD_true, marker = 'o', linestyle='', ms=8, label=name)
    marker_color = line.get_color() #retrieves colour that has been automatically assigned to markers for current group
    ax.plot(xfit, fit(xfit, *popt), linestyle='--', color=marker_color)

    #print fitted doubling time
    print(f"{name} = {np.log(2)/(popt[1]):.3f}")

ax.legend()
plt.xlabel('Timepoint')
plt.ylabel('OD_true')
plt.title('Growth curve and exponential fit')
plt.show()